# Homework 3

# Finite difference methods.

## Explicit Finite Difference

Our goal is to value European options with $V(S, t)$.

### Solving the PDE

We will follow the Black-Scholes assumption that the underlying stock follows this stochastic process:

$$dS_t = rS_t dt + \sigma S_t dW_t$$

Then the price of the European option must satisfy this PDE:

$$\frac{\partial V}{\partial t} + rS \frac{\partial V}{\partial S} + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} - rV = 0$$

In order to solve this PDE, we must have constant coefficients. We can do this through a change of variable, modeling returns instead of the actual stock price.

$$S = e^x$$
$$x = \ln S$$

Then we will get a new value equation $u$ where
$$V(S, t) = V(e^x, t) = u(x, t)$$
$$\frac{\partial V}{\partial t}(t, S) = \frac{\partial u}{\partial t}(t, x)$$

And using Ito's lemma, our original Black-Scholes PDE becomes

$$\frac{\partial u}{\partial t} + \nu\frac{\partial u}{\partial x} + \frac{1}{2}\sigma^2 \frac{\partial^2 u}{\partial x^2} - ru = 0$$

where

$$\nu = r - \frac{\sigma^2}{2}$$

Merton (1973) showed that this PDE, like the heat equation, can be solved analytically and used to value options through the famous Black-Scholes equation. But we can also solve it via a numerical method, the **explicit finite difference** method.

### Discretizing the Domain

This process involves discretizing this equation, and solving it backwards from the payoff at maturity $T$.

This begins by discretizing our domain.

Our domain is $t \in [0, T]$ and $x \in (-\infty, \infty)$.

We will discretize $t$ into $n + 1$ points like so:
$$\Delta t = \frac{T}{n}$$
$$t = \{0, \Delta t, 2 \Delta t, \ldots, n \Delta t\}$$

For $x$, we must set some large boundary instead of using $\infty$, which we will define as $N$. Therefore we will have $2N + 1$ points like so:
$$x = \{-N \Delta x, (-N + 1) \Delta x, \ldots, 0, \Delta x, \ldots, N \Delta x\}$$

The value of $\Delta x$ is technically arbitrary. However, in order for this process to converge, $\Delta x$ must follow

$$\Delta x \geq \sigma \sqrt{3 \Delta t}$$

The time complexity of the explicit algorithm is $O(\Delta x^2 + \Delta t)$. Since we want to minimize the time complexity, the best choice of $\Delta x$ is in practice always $\sigma \sqrt{3\Delta t}$.

### Discretizing the Derivatives

For the explicit finite difference method, there are four points we will need. These are
- $u_{i+1, j+1}$
- $u_{i+1, j}$
- $u_{i+1, j-1}$
- $u_{i, j}$

And there are three derivatives we are trying to calculate
$$\frac{\partial u}{\partial t}, \frac{\partial u}{\partial x}, \frac{\partial^2 u}{\partial x^2}$$

We can use the limit equation for derivatives to describe finite difference for the first-order derivatives.

$$u'(x) = \lim_{h \rightarrow 0}  \frac{u(x + h) - u(x)}{h}$$

And we can also use Taylor expansion to get the limit equation for second-order derivatives in terms of the first-order equation.

$$u''(x) = \lim_{h \rightarrow 0} \frac{u(x+h) - 2u(x) + u(x-h)}{h^2}$$

The derivative with respect to $t$ most neatly fits into this paradigm. If we define
$$h = \Delta t$$
then we get
$$\frac{\partial u}{\partial t} = \frac{u_{i+1, j} - u_{i, j}}{\Delta t}$$

For the first-order derivative with respect to $x$, because we are calculating these values with respect to $u_{i, j}$, we don't want to bias it up or down, so we will use the above and below point and then average them.
$$h = \Delta x$$
$$\frac{\partial u}{\partial x} = \frac{u_{i+1,j+1} - u_{i+1,j-1}}{2\Delta x}$$

And for the second-order derivative with respect to $x$ we will use the corresponding limit equation.

$$h = \Delta x$$
$$\frac{\partial^2 u}{\partial x^2} = \frac{u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1}}{\Delta x^2}$$

### The Discretized Equation

$$\frac{\partial u}{\partial t} + \nu\frac{\partial u}{\partial x} + \frac{1}{2}\sigma^2 \frac{\partial^2 u}{\partial x^2} - ru = 0$$

Substituting back these finite differences into our original equation, we get

$$\frac{u_{i+1, j} - u_{i, j}}{\Delta t} + \nu \frac{u_{i+1,j+1} - u_{i+1,j-1}}{2\Delta x} + \frac{1}{2} \sigma^2 \frac{u_{i+1,j+1} - 2u_{i+1,j} + u_{i+1,j-1}}{\Delta x^2} - r u_{i+1, j} = 0$$

Expand out the equation.
$$\frac{u_{i+1, j}}{\Delta t} - \frac{u_{i, j}}{\Delta t} + \frac{\nu}{2\Delta x} u_{i+1, j+1} - \frac{\nu}{2\Delta x} u_{i+1, j-1} + \frac{\sigma^2}{2\Delta x^2} u_{i+1, j+1} - \frac{\sigma^2}{\Delta x^2} u_{i+1, j} + \frac{\sigma^2}{2 \Delta x^2} h - ru_{i+1, j} = 0$$

Rearrange to solve for $u_{i, j}$.
$$u_{i, j} = \Delta t \left( \frac{\sigma^2}{2\Delta x^2} u_{i+1, j+1} + \frac{\nu}{2\Delta x} u_{i+1, j+1} \right) + \Delta t \left( \frac{\sigma^2}{2\Delta x^2} u_{i+1, j-1} - \frac{\nu}{2\Delta x} u_{i+1, j-1} \right) + u_{i+1, j} - \Delta t \frac{\sigma^2}{\Delta x^2} u_{i+1, j} - ru_{i+1, j} \Delta t$$

Factor out the probabilities.
$$u_{i, j} = p_u u_{i+1, j+1} + p_m u_{i+1, j} + p_d u_{i+1, j-1}$$

$$p_u = \Delta t \left( \frac{\sigma^2}{2\Delta x^2} + \frac{\nu}{2\Delta x} \right)$$
$$p_m = 1 - \Delta t \frac{\sigma^2}{\Delta x^2} - r \Delta t$$
$$p_d = \Delta t \left( \frac{\sigma^2}{2\Delta x^2} - \frac{\nu}{2\Delta x} \right)$$

### Valuation

To carry out the valuation, we will follow these steps.

1. Initialize the following constants:
    - $K$
    - $T$
    - $S$
    - $\sigma$
    - $r$
    - $\delta$
    - $n$
    - $N$
    - $dt$
    - $dx$
    - $\nu$
    - $p_u$
    - $p_m$
    - $p_d$
2. Create a vector of asset prices at maturity.
3. Initialize option values at maturity based on the option payoff formula.
4. Step backwards through the lattice by solving the discretized equation for each point in each time step based on the three points in the next time step.
5. For the boundary conditions, initialize them based on the option type.
6. Return the value at (0, 0)

# Nodes Required for Error

## Explicit Finite Difference

For explicit finite difference.